In [1]:
import numpy as np
import pprint
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv

In [2]:
pp = pprint.PrettyPrinter(indent=2)
env = GridworldEnv()

In [3]:
def value_iteration(env, theta=0.0001, discount_factor=1.0):
    """
    Value Iteration Algorithm.
    
    Args:
        env: OpenAI environment. env.P represents the transition probabilities of the environment.
        theta: Stopping threshold. If the value of all states changes less than theta
            in one iteration we are done.
        discount_factor: gamma time discount factor.
        
    Returns:
        A tuple (policy, V) of the optimal policy and the optimal value function.        
    """
    

    V = np.zeros(env.nS)
    policy = np.zeros([env.nS, env.nA])
    gamma = discount_factor
    i = 0
        
    # Implement!
    while True:
        
        delta = 0 # measures Delta V[.]
        for s in range(env.nS):
            Vq = np.zeros(env.nA)

            for a in range(env.nA):

                # Compute q[s,a] for curernt a
                for P,St1,Ra,_ in env.P[s][a]:
                    Vq[a] += P*V[St1]
                Vq[a] = Ra + gamma*Vq[a]
                
            delta = max(delta, abs(V[s]-np.max(Vq)))
            V[s] = np.max(Vq) # Max over q's (No Expectation)
            policy[s] = np.eye(env.nA)[np.argmax(Vq)]
            
        i += 1  
        if delta < theta:           
            return policy, V, i

In [4]:
policy, v, i = value_iteration(env)

print ("Num.Iterations = ",i,"\n")

print("Policy Probability Distribution:")
print(policy)
print("")

print("Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):")
print(np.reshape(np.argmax(policy, axis=1), env.shape))
print("")

print("Value Function:")
print(v)
print("")

print("Reshaped Grid Value Function:")
print(v.reshape(env.shape))
print("")

Num.Iterations =  4 

Policy Probability Distribution:
[[ 1.  0.  0.  0.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  1.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 1.  0.  0.  0.]]

Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):
[[0 3 3 2]
 [0 0 0 2]
 [0 0 1 2]
 [0 1 1 0]]

Value Function:
[ 0. -1. -2. -3. -1. -2. -3. -2. -2. -3. -2. -1. -3. -2. -1.  0.]

Reshaped Grid Value Function:
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]



In [5]:
# Test the value function
expected_v = np.array([ 0, -1, -2, -3, -1, -2, -3, -2, -2, -3, -2, -1, -3, -2, -1,  0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)

# Run Simulation with Computed Policy 

In [6]:
no_episodes = 3

for i_episode in range(no_episodes):
    observation = env.reset() # Start episode    
    
    for t in range(100):
        env.render() # simulate a time step
        print("")
        action = np.argmax(policy[observation]) # sample action from policy distribution
        observation, _, done, _ = env.step(action) # execute sampled action 
        if done:
            env.render() # simulate a time step
            print("\nEpisode finished after {} timesteps\n".format(t+1))
            break
    
env.render(close=True)# Close window

T  o  o  o
o  o  o  o
o  o  o  x
o  o  o  T

T  o  o  o
o  o  o  o
o  o  o  o
o  o  o  x

Episode finished after 1 timesteps

T  o  o  o
o  o  x  o
o  o  o  o
o  o  o  T

T  o  x  o
o  o  o  o
o  o  o  o
o  o  o  T

T  x  o  o
o  o  o  o
o  o  o  o
o  o  o  T

x  o  o  o
o  o  o  o
o  o  o  o
o  o  o  T

Episode finished after 3 timesteps

T  o  o  o
o  o  x  o
o  o  o  o
o  o  o  T

T  o  x  o
o  o  o  o
o  o  o  o
o  o  o  T

T  x  o  o
o  o  o  o
o  o  o  o
o  o  o  T

x  o  o  o
o  o  o  o
o  o  o  o
o  o  o  T

Episode finished after 3 timesteps



# Use Value Iteration for Froze Lake Env. 

In [16]:
import gym
import numpy as np

# Compute V* and PI
env = gym.make('FrozenLake-v0')
theta = 1.0
policy, v, i = value_iteration (env,theta)

print("Reshaped Grid Value Function:")
print(v.reshape([4,4]))
print("")

# Simulate Scenario
no_episodes = 3
for i_episode in range(no_episodes):
    observation = env.reset() # Start episode    
    
    for t in range(100):
        env.render() # simulate a time step
        print("")
        action = np.argmax(policy[observation]) # sample action from policy distribution
        observation, _, done, _ = env.step(action) # execute sampled action 
        if done:
            env.render() # simulate a time step
            print("\nEpisode finished after {} timesteps\n".format(t+1))
            break
    
env.render(close=True)# Close window


[2016-10-17 16:23:13,197] Making new env: FrozenLake-v0


Reshaped Grid Value Function:
[[ 0.          0.          0.14814815  0.37037037]
 [ 0.          0.22222222  0.82716049  1.46090535]
 [ 0.14814815  0.83950617  2.15226337  3.4951989 ]
 [ 0.38271605  1.5473251   3.77777778  4.84087791]]

SFFF
FHFH
FFFH
HFFG


SFFF
FHFH
FFFH
HFFG
  (Left)

SFFF
FHFH
FFFH
HFFG
  (Left)

SFFF
FHFH
FFFH
HFFG
  (Left)

SFFF
FHFH
FFFH
HFFG
  (Down)

SFFF
FHFH
FFFH
HFFG
  (Right)

Episode finished after 5 timesteps

SFFF
FHFH
FFFH
HFFG
  (Right)

SFFF
FHFH
FFFH
HFFG
  (Left)

SFFF
FHFH
FFFH
HFFG
  (Left)

SFFF
FHFH
FFFH
HFFG
  (Left)

SFFF
FHFH
FFFH
HFFG
  (Down)

Episode finished after 4 timesteps

SFFF
FHFH
FFFH
HFFG
  (Down)

SFFF
FHFH
FFFH
HFFG
  (Left)

SFFF
FHFH
FFFH
HFFG
  (Left)

SFFF
FHFH
FFFH
HFFG
  (Left)

SFFF
FHFH
FFFH
HFFG
  (Left)

SFFF
FHFH
FFFH
HFFG
  (Down)

SFFF
FHFH
FFFH
HFFG
  (Down)

SFFF
FHFH
FFFH
HFFG
  (Right)

SFFF
FHFH
FFFH
HFFG
  (Down)

SFFF
FHFH
FFFH
HFFG
  (Down)

Episode finished after 9 timesteps

